<a href="https://colab.research.google.com/github/frankfurtmacmoses/statistics_for_big_data_assignments/blob/main/Assignment_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
from google.colab import files
files.upload()


Saving kaggle.json to kaggle (1).json


{'kaggle (1).json': b'{"username":"frankfurt","key":"0486f407b3ce3583f098ac48742f1872"}'}

In [11]:
!pip install -q kaggle
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json


In [16]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [17]:
!kaggle competitions download -c house-prices-advanced-regression-techniques

house-prices-advanced-regression-techniques.zip: Skipping, found more recently modified local copy (use --force to force download)


In [33]:
!ls /content

 drive						  'kaggle (1).json'   kaggle.json
 house-prices-advanced-regression-techniques.zip   kaggle_data	      sample_data


In [22]:
!pwd

/content


In [28]:
!mkdir /content/drive/MyDrive/kaggle_data


In [34]:
!unzip /content/house-prices-advanced-regression-techniques.zip -d /content/drive/MyDrive/kaggle_data/

Archive:  /content/house-prices-advanced-regression-techniques.zip
  inflating: /content/drive/MyDrive/kaggle_data/data_description.txt  
  inflating: /content/drive/MyDrive/kaggle_data/sample_submission.csv  
  inflating: /content/drive/MyDrive/kaggle_data/test.csv  
  inflating: /content/drive/MyDrive/kaggle_data/train.csv  
